# ガバナンス状態を検証する
データガバナンス機能で作成したデータやGakunin RDMで付与したメタデータを検証、実験の再現性を確認するタスクです。

## 1. 検証画面（dg-web）を表示する

In [ ]:
# 検証画面（dg-web）を表示する
import sys

import panel as pn
from IPython.display import display
from IPython.core.display import Javascript

sys.path.append('../../../../../..')
from library.utils.html.button import create_button
from library.utils.config import message as msg_config

def access_validate():
    button_width = 500
    url = 'https://web.dg.nii.ac.jp/'
    obj = create_button(
        url=url,
        target='_blank',
        msg=msg_config.get('task', 'open_validate'),
        button_width=f'{button_width}px'
    )
    pn.extension()
    display(pn.pane.HTML(obj, width=button_width))
    display(Javascript('IPython.notebook.save_checkpoint();'))

access_validate()

## サブフローメニューへ

サブフローメニューへアクセスするボタンを表示します

In [ ]:
# サブフローメニューへアクセスするボタンを表示
import os
import sys
sys.path.append('../../../../..') # 研究準備
sys.path.append('../../../../../..') # 研究準備以外
from library.task_director import TaskDirector

script_file_name = "validate"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()